In [1]:
# es5.1 & kibana5.1起動
!cd docker && docker-compose up --build -d

Building elasticsearch
Step 1/6 : FROM elasticsearch:5.1
 ---> 3b9ab09f293c
Step 2/6 : RUN bin/elasticsearch-plugin install analysis-icu
 ---> Using cache
 ---> d45f30516d2e
Step 3/6 : RUN bin/elasticsearch-plugin install analysis-kuromoji
 ---> Using cache
 ---> 86b7cb36ca46
Step 4/6 : RUN bin/elasticsearch-plugin install analysis-phonetic
 ---> Using cache
 ---> d8e645b6c1ef
Step 5/6 : EXPOSE 9200
 ---> Using cache
 ---> 90702b10f3da
Step 6/6 : CMD elasticsearch -Etransport.host=0.0.0.0
 ---> Using cache
 ---> 0d58485ca1c1
Successfully built 0d58485ca1c1
Building kibana
Step 1/1 : FROM kibana:5.1
 ---> 244bc3968631
Successfully built 244bc3968631
docker_elasticsearch_1 is up-to-date
docker_kibana_1 is up-to-date


In [2]:
# -*- coding: utf-8 -*-
%reload_ext autoreload
%autoreload 2

import sys
import json
import pprint
sys.path.append("lib/")
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import graphviz as gv

INDEX='my-index'
TYPE_NAME='my-type'

es = Elasticsearch(
    host='localhost',
    port=9200,
    use_ssl=False
)
#es = Elasticsearch()

In [3]:
#2.3.0にあったchar_filters, filters が廃止
def analyzer(index, analyzer, char_filters, tokenizer, filters, text, explain=False):
    analyzer = es.indices.analyze(
        index=index,
        text=text,
        char_filter=char_filters,
        tokenizer=tokenizer,
        filter=filters,
        explain=explain
    )
    return analyzer

Analyzer
==


Tokenizer
-------------

Tokenizers | Elasticsearch Reference [5.1] | Elastic

https://www.elastic.co/guide/en/elasticsearch/reference/5.1/analysis-tokenizers.html

について学ぶ

種類
-------------

- Word Oriented Tokenizers
- Partial Word Tokenizersedit
- Structured Text Tokenizers

がある。

### Word Oriented Tokenizers

* standard
* letter
* lowercase
* whitespace
* uax_url_email
* classic
* thai

### Partial Word Tokenizersedit

* ngram
* edge_ngram

### Structured Text Tokenizers

* keyword
* pattern
* path_hierarchy



Structured Text Tokenizers: keyword
-----------

文字列全体をトークンとして扱う（区切らない）

In [4]:
char_filters = ""
tokenizer = "keyword"
filters = ""

text="I have a pen. I have an apple."
print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "I have a pen. I have an apple."
]


Structured Text Tokenizers: pattern
-----------

正規表現でトークン化する



In [5]:
text="I have a pen. I have an apple. Yes, I have."

char_filters=""
tokenizer = "pattern"
filters=""


print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "I", 
    "have", 
    "a", 
    "pen", 
    "I", 
    "have", 
    "an", 
    "apple", 
    "Yes", 
    "I", 
    "have"
]


### pattern tokenizerその２

`I` で区切ってみる


In [6]:
body="""
{
  "settings": {
    "analysis": {
      "tokenizer": {
        "I_tokenizer": {
          "type": "pattern",
          "pattern": "I"
        }
      }
    }
  }
}
"""

es.indices.close(index=INDEX)
es.indices.put_settings(index=INDEX, body=body)
es.indices.open(index=INDEX)


{u'acknowledged': True}

In [7]:
text="I have a pen. I have an apple. Yes, I have."

char_filters=[]
tokenizer = "I_tokenizer"
filters=[]

print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    " have a pen. ", 
    " have an apple. Yes", 
    " ", 
    " have."
]


In [8]:
# 前後の空白削除は trim token filter を使う
text="I have a pen. I have an apple. Yes, I have."

char_filters=[]
tokenizer = "I_tokenizer"
filters=["trim"]

print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "have a pen.", 
    "have an apple. Yes", 
    "", 
    "have."
]


Structured Text Tokenizers: path_hierarchy
------------

In [9]:
text="/path/to/command"

char_filters=[]
tokenizer = "path_hierarchy"
filters=[]

print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "/path", 
    "/path/to", 
    "/path/to/command"
]


In [10]:
text="C:\path\to\command"

char_filters=[]
tokenizer = "path_hierarchy"
filters=[]

print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "C:\\path\to\\command"
]
